In [1]:
import os
os.environ['ESMFMKFILE'] = '/home/malasawedah/.conda/envs/test2/lib/esmf.mk'

# Now you can import the ESMF library
#import ESMF


In [2]:
import datetime
import geopandas as gpd
import pandas as pd
import xarray as xr
import numpy as np
#import xvec
import shapely
from shapely import Point, box
import esmpy
import xagg as xa
import rioxarray

In [3]:
#!pip install dask

In [4]:
aoi = gpd.read_file('/mnt/CEPH_PROJECTS/ScaleAgData/03_Ancillary_Data/LAFIS_zonal_stats_prato_stabile_1000.shp')
grid5 = pd.read_csv('/mnt/CEPH_PROJECTS/ScaleAgData/02_EO_Data/s1/polygons_grid_5')
#chunks = {'t': 1, 'x': 100, 'y': 100}
#s1 = xr.open_dataset(f'/mnt/CEPH_PROJECTS/ScaleAgData/02_EO_Data/s1/latest/GRD_luca/asc/grid5/S1_grid5_1/openEO.nc', chunks= chunks)
s1 = xr.open_dataset(f'/mnt/CEPH_PROJECTS/ScaleAgData/02_EO_Data/s1/latest/GRD_luca/asc/grid5/S1_grid5_1/openEO.nc')

In [5]:
s1

<xarray.Dataset>
Dimensions:  (t: 12, x: 3764, y: 2282)
Coordinates:
  * t        (t) datetime64[ns] 2021-03-03 2021-03-04 ... 2021-04-02 2021-04-03
  * x        (x) float64 6.478e+05 6.478e+05 6.478e+05 ... 6.854e+05 6.854e+05
  * y        (y) float64 5.17e+06 5.17e+06 5.17e+06 ... 5.147e+06 5.147e+06
Data variables:
    crs      |S1 ...
    VV       (t, y, x) float32 ...
    VH       (t, y, x) float32 ...
Attributes:
    Conventions:  CF-1.9
    institution:  openEO platform - Geotrellis backend: 0.15.1a1
    description:  
    title:

In [6]:
s1.rio.write_crs("EPSG:32632", inplace=True)

<xarray.Dataset>
Dimensions:  (t: 12, x: 3764, y: 2282)
Coordinates:
  * t        (t) datetime64[ns] 2021-03-03 2021-03-04 ... 2021-04-02 2021-04-03
  * x        (x) float64 6.478e+05 6.478e+05 6.478e+05 ... 6.854e+05 6.854e+05
  * y        (y) float64 5.17e+06 5.17e+06 5.17e+06 ... 5.147e+06 5.147e+06
    crs      int64 0
Data variables:
    VV       (t, y, x) float32 ...
    VH       (t, y, x) float32 ...
Attributes:
    Conventions:  CF-1.9
    institution:  openEO platform - Geotrellis backend: 0.15.1a1
    description:  
    title:

In [7]:
s1_wgs84 = s1.rio.reproject("EPSG:4326")

In [8]:
s1_wgs84

<xarray.Dataset>
Dimensions:  (x: 4159, y: 1788, t: 12)
Coordinates:
  * x        (x) float64 10.92 10.92 10.92 10.92 ... 11.42 11.42 11.42 11.42
  * y        (y) float64 46.67 46.67 46.67 46.67 ... 46.46 46.46 46.46 46.46
  * t        (t) datetime64[ns] 2021-03-03 2021-03-04 ... 2021-04-02 2021-04-03
    crs      int64 0
Data variables:
    VV       (t, y, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    VH       (t, y, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan
Attributes:
    Conventions:  CF-1.9
    institution:  openEO platform - Geotrellis backend: 0.15.1a1
    description:  
    title:

In [9]:
grid5['geometry'] = grid5['geometry'].apply(lambda x: shapely.wkt.loads(x))
gdf_grid5 = gpd.GeoDataFrame(grid5, geometry='geometry')
gdf_grid5.crs = 'EPSG:4326'
gdf_grid5.head(3)

,Unnamed: 0,ID,FID,CUAA,CODE,DESCR_IT,DESCR_DE,area,dem_mean,dem_median,slo_mean,slo_median,asp_mean,asp_median,north_mean,north_medi,dem_class,geometry
0,0,2894152,PRGMFR75E13F132Z-1688030,PRGMFR75E13F132Z,AP2,Prato stabile,Wiese (Dauerwiese),56,1301.867221,1301.705200,17.897289,17.806307,85.131834,84.665092,1.000000,1.0,1300.0,"POLYGON ((11.02662 46.46839, 11.02669 46.46830..."
1,1,2347633,RNRGTH80H18F132V-59,RNRGTH80H18F132V,AP2,Prato stabile,Wiese (Dauerwiese),31147,1302.440111,1302.553589,22.939459,22.398680,145.304115,153.931000,0.043887,0.0,1300.0,"POLYGON ((10.94554 46.65611, 10.94556 46.65604..."
2,2,2970168,WSSCRS85L19A952F-2158250,WSSCRS85L19A952F,AP2,Prato stabile,Wiese (Dauerwiese),842,1318.083754,1317.940063,12.269650,12.262682,228.961556,228.782425,0.000000,0.0,1300.0,"POLYGON ((11.13220 46.49439, 11.13249 46.49438..."


In [10]:
#chunks = {'t': 1, 'x': 100, 'y': 100}
#s1 = xr.open_dataset(f'/mnt/CEPH_PROJECTS/ScaleAgData/02_EO_Data/s1/latest/GRD_luca/asc/grid5/S1_grid5_1/openEO.nc', chunks= chunks)
#s1_wgs84 = s1_wgs84.chunk({'t': 1, 'x': 50, 'y': 50})


In [19]:
%%time
weightmap = xa.pixel_overlaps(s1_wgs84,gdf_grid5)

creating polygons for each pixel...
lat/lon bounds not found in dataset; they will be created.
calculating overlaps between pixels and output polygons...
success!
CPU times: user 2min 30s, sys: 2.68 s, total: 2min 32s
Wall time: 2min 32s


In [20]:
ds = s1_wgs84
wm = weightmap
impl='for_loop'
silent=False

In [21]:
# Make sure pixel_overlaps was correctly run if using dot product
if (impl=='dot_product') and (wm.overlap_da is None):
    raise ValueError("no 'overlap_da' was found in the `wm` input - since you're using the dot product implementation, "+
                     "make sure to run `pixel_overlaps()` with `impl='dot_product'` to avoid this error.")

# Turn into dataset if dataarray
if type(ds)==xr.core.dataarray.DataArray:
  if ds.name is None:
    warnings.warn('An unnamed xr.DataArray was inputted instead of a xr.Dataset; the output variable will be "var"')
    ds = ds.to_dataset(name='var')
  else:
    ds = ds.to_dataset()


# Run ds through fix_ds (to fix lat/lon names, lon coords)
ds = fix_ds(ds)

# Stack 
ds = ds.stack(loc=('lat','lon'))

In [22]:
ds

<xarray.Dataset>
Dimensions:  (t: 12, loc: 7436292)
Coordinates:
  * t        (t) datetime64[ns] 2021-03-03 2021-03-04 ... 2021-04-02 2021-04-03
    crs      int64 0
  * loc      (loc) object MultiIndex
  * lat      (loc) float64 46.46 46.46 46.46 46.46 ... 46.67 46.67 46.67 46.67
  * lon      (loc) float64 10.92 10.92 10.92 10.92 ... 11.42 11.42 11.42 11.42
Data variables:
    VV       (t, loc) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    VH       (t, loc) float32 nan nan nan nan nan nan ... nan nan nan nan nan
Attributes:
    Conventions:  CF-1.9
    institution:  openEO platform - Geotrellis backend: 0.15.1a1
    description:  
    title:

In [23]:
ds = subset_find(ds,wm.source_grid)

adjusting grid... (this may happen because only a subset of pixels were used for aggregation for efficiency - i.e. [subset_bbox=True] in xa.pixel_overlaps())
grid adjustment successful


In [24]:
if type(wm.weights) == pd.core.series.Series:
    weights = np.array([float(k) for k in wm.weights])
else:
    if wm.weights != 'nowghts':
        warnings.warn('wm.weights is: \n '+print(wm.weights)+
                        ', \n which is not a supported weight vector (in a pandas series) '+
                        'or "nowghts" as a string. Assuming no weights are included...')
    if impl=='dot_product':
        weights = np.ones((len(wm.overlap_da['loc'])))
    elif impl=='for_loop':
        weights = np.ones((len(wm.source_grid['lat'])))

In [25]:
%%time
for var in ds:
    # Process for every variable that has locational information, but isn't a 
    # bound variable
    if ('bnds' not in ds[var].dims) & ('loc' in ds[var].dims):
        print(var)
        if not silent:
            print('aggregating '+var+'...')
        # Create the column for the relevant variable
        wm.agg[var] = None

        # Get weighted average of variable based on pixel overlap + other weights
        for poly_idx in wm.agg.poly_idx:
            # Get average value of variable over the polygon; weighted by 
            # how much of the pixel area is in the polygon, and by (optionally)
            # a separate gridded weight. This if statement checks 
            # - if the weights output for this polygon is just "np.nan", which 
            #   indicates there's no overlap between the pixel grid and polygons
            # - [this has been pushed down a few lines] or, if all the pixels in 
            #   the grid have just nan values for this variable
            # in both cases; the "aggregated variable" is just a vector of nans. 
            if not np.isnan(wm.agg.iloc[poly_idx,:].pix_idxs).all():
                # Get the dimensions of the variable that aren't "loc" (location)
                other_dims = [k for k in np.atleast_1d(ds[var].dims) if k != 'loc']
                # Check first if any nans are "complete" (meaning that a pixel 
                # either has values for each step, or nans for each step - if
                # there are random nans within a pixel, throw a warning)
                if not xr.Dataset.equals(np.isnan(ds[var].isel(loc=wm.agg.iloc[poly_idx,:].pix_idxs)).any(other_dims),
                                          np.isnan(ds[var].isel(loc=wm.agg.iloc[poly_idx,:].pix_idxs)).all(other_dims)):
                    warnings.warn('One or more of the pixels in variable '+var+' have nans in them in the dimensions '+
                                   ', '.join([k for k in ds[var].dims if k != 'loc'])+
                                   '. The code can currently only deal with pixels for which the '+
                                   '*entire* pixel has nan values in all dimensions, however there is currently no '+
                                   ' support for data in which pixels have only some nan values. The aggregation '+
                                   'calculation is likely incorrect.')

                if not np.isnan(ds[var].isel(loc=wm.agg.iloc[poly_idx,:].pix_idxs)).all(): 
                    # Get relative areas for the pixels overlapping with this Polygon
                    tmp_areas = np.atleast_1d(np.squeeze(wm.agg.iloc[poly_idx,:].rel_area))
                    # Replace overlapping pixel areas with nans if the corresponding pixel
                    # is only composed of nans
                    tmp_areas[np.array(np.isnan(ds[var].isel(loc=wm.agg.iloc[poly_idx,:].pix_idxs)).all(other_dims).values)] = np.nan
                    # Calculate the normalized area+weight of each pixel (taking into account
                    # nans)
                    normed_areaweights = normalize(tmp_areas*weights[wm.agg.iloc[poly_idx,:].pix_idxs],drop_na=True)

                    # Take the weighted average of all the pixel values to calculate the 
                    # aggregated value for the shapefile
                    wm.agg.loc[poly_idx,var] = [[((ds[var].isel(loc=wm.agg.iloc[poly_idx,:].pix_idxs)*
                                                     normed_areaweights).
                                                    sum('loc')).values]]
                    
                    #wm.agg.loc[poly_idx,var] = [[((ds[var].isel(loc=wm.agg.iloc[poly_idx,:].pix_idxs)*
                    #                                 normed_areaweights).
                     #                               sum('loc', skipna=True)).values]]

                else:
                    wm.agg.loc[poly_idx,var] = [[(ds[var].isel(loc=0)*np.nan).values]]

            else:
                wm.agg.loc[poly_idx,var] = [[(ds[var].isel(loc=0)*np.nan).values]]
    #break
# Put in class format
agg_out = aggregated(agg=wm.agg,source_grid=wm.source_grid,
                     geometry=wm.geometry,ds_in=ds,weights=wm.weights)


VV
aggregating VV...


<timed exec>:29: UserWarning: One or more of the pixels in variable VV have nans in them in the dimensions t. The code can currently only deal with pixels for which the *entire* pixel has nan values in all dimensions, however there is currently no  support for data in which pixels have only some nan values. The aggregation calculation is likely incorrect.
<timed exec>:29: UserWarning: One or more of the pixels in variable VV have nans in them in the dimensions t. The code can currently only deal with pixels for which the *entire* pixel has nan values in all dimensions, however there is currently no  support for data in which pixels have only some nan values. The aggregation calculation is likely incorrect.
<timed exec>:29: UserWarning: One or more of the pixels in variable VV have nans in them in the dimensions t. The code can currently only deal with pixels for which the *entire* pixel has nan values in all dimensions, however there is currently no  support for data in which pixels ha

VH
aggregating VH...


<timed exec>:29: UserWarning: One or more of the pixels in variable VH have nans in them in the dimensions t. The code can currently only deal with pixels for which the *entire* pixel has nan values in all dimensions, however there is currently no  support for data in which pixels have only some nan values. The aggregation calculation is likely incorrect.
<timed exec>:29: UserWarning: One or more of the pixels in variable VH have nans in them in the dimensions t. The code can currently only deal with pixels for which the *entire* pixel has nan values in all dimensions, however there is currently no  support for data in which pixels have only some nan values. The aggregation calculation is likely incorrect.
<timed exec>:29: UserWarning: One or more of the pixels in variable VH have nans in them in the dimensions t. The code can currently only deal with pixels for which the *entire* pixel has nan values in all dimensions, however there is currently no  support for data in which pixels ha

CPU times: user 778 ms, sys: 3.97 ms, total: 782 ms
Wall time: 795 ms


<timed exec>:29: UserWarning: One or more of the pixels in variable VH have nans in them in the dimensions t. The code can currently only deal with pixels for which the *entire* pixel has nan values in all dimensions, however there is currently no  support for data in which pixels have only some nan values. The aggregation calculation is likely incorrect.
<timed exec>:29: UserWarning: One or more of the pixels in variable VH have nans in them in the dimensions t. The code can currently only deal with pixels for which the *entire* pixel has nan values in all dimensions, however there is currently no  support for data in which pixels have only some nan values. The aggregation calculation is likely incorrect.
<timed exec>:29: UserWarning: One or more of the pixels in variable VH have nans in them in the dimensions t. The code can currently only deal with pixels for which the *entire* pixel has nan values in all dimensions, however there is currently no  support for data in which pixels ha

In [28]:
ds_out = agg_out.to_dataset()
ds_out

/home/malasawedah/.conda/envs/test2/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1734: RuntimeWarning: invalid value encountered in subtract
  np.subtract(arr, avg, out=arr, casting='unsafe', where=where)
/home/malasawedah/.conda/envs/test2/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1734: RuntimeWarning: invalid value encountered in subtract
  np.subtract(arr, avg, out=arr, casting='unsafe', where=where)
/home/malasawedah/.conda/envs/test2/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1734: RuntimeWarning: invalid value encountered in subtract
  np.subtract(arr, avg, out=arr, casting='unsafe', where=where)
/home/malasawedah/.conda/envs/test2/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1734: RuntimeWarning: invalid value encountered in subtract
  np.subtract(arr, avg, out=arr, casting='unsafe', where=where)
/home/malasawedah/.conda/envs/test2/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1734: RuntimeWarning: invalid value encountered i

<xarray.Dataset>
Dimensions:     (poly_idx: 101, t: 12)
Coordinates:
  * poly_idx    (poly_idx) int64 0 1 2 3 4 5 6 7 8 ... 93 94 95 96 97 98 99 100
  * t           (t) datetime64[ns] 2021-03-03 2021-03-04 ... 2021-04-03
Data variables: (12/19)
    Unnamed: 0  (poly_idx) int64 0 1 2 3 4 5 6 7 8 ... 93 94 95 96 97 98 99 100
    ID          (poly_idx) int64 2894152 2347633 2970168 ... 2060657 2614883
    FID         (poly_idx) object 'PRGMFR75E13F132Z-1688030' ... 'NTRSFR65C25...
    CUAA        (poly_idx) object 'PRGMFR75E13F132Z' ... 'NTRSFR65C25I431Z'
    CODE        (poly_idx) object 'AP2' 'AP2' 'AP2' 'AP2' ... 'AP2' 'AP2' 'AP2'
    DESCR_IT    (poly_idx) object 'Prato stabile' ... 'Prato stabile'
    ...          ...
    asp_median  (poly_idx) float64 84.67 153.9 228.8 139.9 ... 198.7 60.5 158.1
    north_mean  (poly_idx) float64 1.0 0.04389 0.0 0.0 ... 0.0 0.009091 1.0 0.0
    north_medi  (poly_idx) float64 1.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 1.0 0.0
    dem_class   (poly_idx) float64 1.3e+03 1.3e+03 1.3e+03 ... 600.0 1.2e+03
    VV          (poly_idx, t) float64 0.0 0.05796 0.0 ... 0.0466 0.0 0.0532
    VH          (poly_idx, t) float64 0.0 0.01789 0.0 ... 0.01388 0.0 0.01195